# Course Creation Example

This notebook demonstrates how to use the `CourseCreator` to generate course ideas and outlines.

## Import and instantiate the CourseCreator class

Initialize a `CourseCreator` instance. All course creation is performed through this object.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

# Check the key
key = os.getenv("OPENAI_API_KEY")
print(f"Key: {key[:8]}...{key[-4:]}")
# Test a simple API call
client = OpenAI()
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # cheapest model
        messages=[{"role": "user", "content": "Say hello"}],
        max_tokens=5
    )
    print(f"Success! Response: {response.choices[0].message.content}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
from openai import OpenAI
client = OpenAI()

models_to_test = ["gpt-4o", "o3-mini", "gpt-4.1"]

for model in models_to_test:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Hi"}],
            #max_tokens=5
        )
        print(f"✓ {model}: works")
    except Exception as e:
        print(f"✗ {model}: {e}")

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

key = os.getenv("GOOGLE_API_KEY")
if key:
    print(f"Key: {key[:8]}...{key[-4:]}")

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=key)
    response = llm.invoke("Say hello")
    print(f"Success! Response: {response.content}")
else:
    print("No GOOGLE_API_KEY found in .env")
    print("Get one free at: https://aistudio.google.com/apikey")

In [ ]:
from denario.course import CourseCreator

creator = CourseCreator(project_dir="./course_project", clear_project_dir=False)

In [ ]:
import denario.prompts.course_outline as outline_prompts
import denario.prompts.course_idea as idea_prompts

print("--- Course Idea Prompt ---")
print(idea_prompts.course_idea_planner_prompt)
print("\n--- Course Outline Prompt ---")
print(outline_prompts.course_outline_researcher_prompt)

## Set the course topic, audience, and duration

Specify the subject you want to teach, who the course is for, and how long it will be.

In [ ]:
creator.set_topic(
    topic="ChatGPT Professional Proficiency: From Basics to Advanced Features",
    audience="Professionals in different fields seeking deep proficiency without coding. They want to go from basics in LLMs such as prompt and context to medium-level features. They want to explore a large set of tools currently offered by ChatGPT. ",
    duration="24 hours"
)

creator.show_topic()

## Generate a course idea

Use the `idea_maker` and `idea_hater` agents to brainstorm and refine course ideas.

In [ ]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
#creator.generate_idea()
creator.generate_idea(
    idea_maker_model="gpt-5",
    idea_hater_model="gpt-5-mini",
    planner_model="gpt-5",
    plan_reviewer_model="gpt-5-mini",
    orchestration_model="gpt-5",
    formatter_model="gpt-5-mini",
)
# Original models (require higher OpenAI tier):
# creator.generate_idea(
#     idea_maker_model="gpt-4o",
#     idea_hater_model="o3-mini",
#     planner_model="gpt-4o",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_idea()

In [ ]:
# Read and display ideas in sequence to see how they developed
import json
from pathlib import Path
from IPython.display import display, Markdown

# Get the run directory from the creator
run_dir = Path(creator.work_dir)
control_dir = run_dir / "idea_generation_output" / "control"

# Read ideas JSON files (snapshots at each idea_maker step)
ideas_files = sorted(control_dir.glob("ideas_*.json"))

print(f"Found {len(ideas_files)} idea snapshots in {control_dir}\n")

step_map = {0: "Step 1: Initial ideas (5 generated)", 
            1: "Step 3: Improved 2 ideas", 
            2: "Step 5: Best idea selected", 
            3: "Step 6: Final formatted idea"}

for i, ideas_file in enumerate(ideas_files):
    display(Markdown(f"---\n## {step_map.get(i, f'Snapshot {i+1}')}"))
    display(Markdown(f"*File: {ideas_file.name}*"))
    
    with open(ideas_file, 'r') as f:
        ideas = json.load(f)
    
    if isinstance(ideas, list):
        for j, idea in enumerate(ideas):
            # Truncate long ideas for readability
            preview = idea[:3000] + "\n\n...[truncated]" if len(idea) > 1500 else idea
            display(Markdown(f"### Idea {j+1}"))
            print(preview)
            print()
    else:
        print(ideas)

In [ ]:
# View idea_hater critiques (steps 2 and 4)
chats_dir = control_dir / "chats"

for step in [2, 4]:
    critique_file = chats_dir / f"chat_output_idea_hater_step_{step}.json"
    if critique_file.exists():
        display(Markdown(f"---\n## Step {step}: idea_hater critique"))
        with open(critique_file, 'r') as f:
            critique = json.load(f)
        # Handle both string and other formats
        critique_str = str(critique)
        preview = critique_str[:6000] + "\n\n...[truncated]" if len(critique_str) > 6000 else critique_str
        print(preview)

## Generate a course outline

Use the `researcher` agent to create a detailed course outline with modules, learning goals, and takeaways.

In [ ]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
#creator.generate_outline()
creator.generate_outline(
    researcher_model="gpt-5",
    planner_model="gpt-5",
    plan_reviewer_model="gpt-5-mini",
    orchestration_model="gpt-5",
    formatter_model="gpt-5-mini", 
)
# Original models (require higher OpenAI tier):
# creator.generate_outline(
#     researcher_model="gpt-4.1-2025-04-14",
#     planner_model="gpt-4.1-2025-04-14",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_outline()

## Alternative: Set idea manually

If you already have a course idea, you can set it directly and skip the idea generation step.

In [ ]:
creator.set_idea(idea=r"""
 Course Title: ChatGPT Premium Unlocked: Tools and Techniques for Business Excellence
 
- This course offers a comprehensive exploration of the advanced tools available in the ChatGPT premium seat, focusing on features like execution flows and custom instructions.
- Participants will engage in interactive workshops and analyze real-world business case studies to enhance practical relevance and applicability.
- Each module includes in the end 1 or 2 practical exercises to apply the tools learned.
- The course inlcudes some ethical AI practices, providing examples directly linked to business implications to ensure responsible usage.
- Scaffolding steps, such as a mini project or simulation exercise, are included to facilitate the transition from tool exploration to practical application.
- Workshop segments are mapped to specific business outcomes, such as increased operational efficiency, to clearly measure the learning impact and ensure actionable insights.

 """)


creator.generate_outline()
creator.show_outline()

## Full workflow in one call

Alternatively, run the entire workflow (idea + outline generation) with a single method call.

In [ ]:
# creator2 = CourseCreator(project_dir="./another_course", clear_project_dir=True)
# creator2.set_topic(
#     topic="Data Visualization with Python",
#     audience="Data analysts familiar with Excel",
#     duration="1 day (4 hours)"
# )
# creator2.run()  # Generates both idea and outline
# creator2.show_outline()